# Pattern 7: Graphs

## Overview

Graphs are data structures that model relationships between objects. They consist of vertices (nodes) connected by edges.

**When to use:**
- Network connectivity problems
- Finding paths between nodes
- Social networks, maps, dependencies
- Scheduling and ordering problems

**Key Insight:** Many problems can be modeled as graph traversal or graph search problems.

**Common Patterns:** DFS, BFS, Topological Sort, Union Find

---

## Graph Fundamentals

### Graph Types

1. **Directed vs Undirected**
   - Directed: Edges have direction (A → B)
   - Undirected: Edges are bidirectional (A ↔ B)

2. **Weighted vs Unweighted**
   - Weighted: Edges have costs/weights
   - Unweighted: All edges equal weight

3. **Connected vs Disconnected**
   - Connected: Path exists between all nodes
   - Disconnected: Some nodes unreachable

4. **Cyclic vs Acyclic**
   - Cyclic: Contains cycles
   - Acyclic: No cycles (DAG = Directed Acyclic Graph)

### Graph Representations

1. **Adjacency List** (most common)
   ```python
   graph = {
       0: [1, 2],
       1: [2],
       2: [3],
       3: []
   }
   ```

2. **Adjacency Matrix**
   ```python
   graph = [
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0]
   ]
   ```

3. **Edge List**
   ```python
   edges = [(0,1), (0,2), (1,2), (2,3)]
   ```

In [ ]:
from collections import defaultdict, deque

# Helper function to build graph from edges
def build_graph(n, edges, directed=False):
    """Build adjacency list from edges."""
    graph = defaultdict(list)
    
    for u, v in edges:
        graph[u].append(v)
        if not directed:
            graph[v].append(u)
    
    return graph

# Example graph
edges = [(0, 1), (0, 2), (1, 2), (2, 3), (2, 4)]
graph = build_graph(5, edges)

print("Graph (adjacency list):")
for node, neighbors in graph.items():
    print(f"  {node} → {neighbors}")

---

## Pattern 1: Depth-First Search (DFS)

DFS explores as deep as possible before backtracking. Uses recursion or stack.

**Time**: O(V + E) where V = vertices, E = edges  
**Space**: O(V) for visited set + O(V) for recursion stack

**When to use:**
- Finding connected components
- Cycle detection
- Path finding
- Topological sort

In [ ]:
def dfs_recursive(graph, start, visited=None):
    """
    DFS using recursion.
    Time: O(V + E), Space: O(V)
    """
    if visited is None:
        visited = set()
    
    visited.add(start)
    result = [start]
    
    for neighbor in graph[start]:
        if neighbor not in visited:
            result.extend(dfs_recursive(graph, neighbor, visited))
    
    return result

def dfs_iterative(graph, start):
    """
    DFS using stack (iterative).
    Time: O(V + E), Space: O(V)
    """
    visited = set()
    stack = [start]
    result = []
    
    while stack:
        node = stack.pop()
        
        if node not in visited:
            visited.add(node)
            result.append(node)
            
            # Add neighbors to stack (in reverse for same order as recursive)
            for neighbor in reversed(graph[node]):
                if neighbor not in visited:
                    stack.append(neighbor)
    
    return result

# Example
graph = build_graph(5, [(0,1), (0,2), (1,3), (2,4)])

print("DFS Traversal:")
print(f"  Recursive:  {dfs_recursive(graph, 0)}")
print(f"  Iterative:  {dfs_iterative(graph, 0)}")

### Visualization: DFS Process

In [ ]:
def dfs_visual(graph, start):
    """Visual walkthrough of DFS."""
    visited = set()
    result = []
    
    def dfs(node, depth=0):
        indent = "  " * depth
        print(f"{indent}Visit node {node}")
        
        visited.add(node)
        result.append(node)
        print(f"{indent}Visited: {sorted(visited)}")
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                print(f"{indent}Explore neighbor {neighbor}")
                dfs(neighbor, depth + 1)
            else:
                print(f"{indent}Skip {neighbor} (already visited)")
        
        print(f"{indent}Backtrack from {node}")
    
    print("DFS Traversal:\n")
    dfs(start)
    print(f"\nTraversal order: {result}")
    return result

graph = build_graph(5, [(0,1), (0,2), (1,3), (2,4)])
dfs_visual(graph, 0)

---

## Pattern 2: Breadth-First Search (BFS)

BFS explores level by level using a queue.

**Time**: O(V + E)  
**Space**: O(V) for queue and visited set

**When to use:**
- Shortest path in unweighted graph
- Level-by-level traversal
- Finding minimum steps

In [ ]:
def bfs(graph, start):
    """
    BFS traversal.
    Time: O(V + E), Space: O(V)
    """
    visited = set([start])
    queue = deque([start])
    result = []
    
    while queue:
        node = queue.popleft()
        result.append(node)
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
    
    return result

# Example
graph = build_graph(5, [(0,1), (0,2), (1,3), (2,4)])
print("BFS Traversal:", bfs(graph, 0))

### Visualization: BFS Process

In [ ]:
def bfs_visual(graph, start):
    """Visual walkthrough of BFS."""
    visited = set([start])
    queue = deque([start])
    result = []
    step = 0
    
    print("BFS Traversal:\n")
    print(f"Initial: queue=[{start}], visited={{{start}}}\n")
    
    while queue:
        node = queue.popleft()
        result.append(node)
        step += 1
        
        print(f"Step {step}: Process node {node}")
        print(f"  Queue before: {list(queue)}")
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
                print(f"  Add neighbor {neighbor} to queue")
            else:
                print(f"  Skip {neighbor} (already visited)")
        
        print(f"  Queue after:  {list(queue)}")
        print(f"  Visited: {sorted(visited)}\n")
    
    print(f"Traversal order: {result}")
    return result

graph = build_graph(5, [(0,1), (0,2), (1,3), (2,4)])
bfs_visual(graph, 0)

---

## Example 1: Number of Connected Components

**Problem**: Find number of connected components in undirected graph.

**Approach**: Run DFS/BFS from each unvisited node.

In [ ]:
def count_components(n, edges):
    """
    Count connected components using DFS.
    Time: O(V + E), Space: O(V + E)
    """
    graph = build_graph(n, edges)
    visited = set()
    count = 0
    
    def dfs(node):
        visited.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor)
    
    for node in range(n):
        if node not in visited:
            dfs(node)
            count += 1
    
    return count

# Example: 3 components: {0,1}, {2,3}, {4}
edges = [(0,1), (2,3)]
n = 5
result = count_components(n, edges)
print(f"Number of components: {result}")

# Example: 1 component (all connected)
edges2 = [(0,1), (1,2), (2,3)]
result2 = count_components(4, edges2)
print(f"Number of components: {result2}")

---

## Example 2: Shortest Path (BFS)

**Problem**: Find shortest path between two nodes in unweighted graph.

**Key Insight**: BFS naturally finds shortest path in unweighted graphs.

In [ ]:
def shortest_path(graph, start, end):
    """
    Find shortest path using BFS.
    Time: O(V + E), Space: O(V)
    """
    if start == end:
        return [start]
    
    visited = {start}
    queue = deque([(start, [start])])  # (node, path)
    
    while queue:
        node, path = queue.popleft()
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                new_path = path + [neighbor]
                
                if neighbor == end:
                    return new_path
                
                visited.add(neighbor)
                queue.append((neighbor, new_path))
    
    return None  # No path exists

# Example
graph = build_graph(6, [(0,1), (0,2), (1,3), (2,3), (3,4), (4,5)])
path = shortest_path(graph, 0, 5)
print(f"Shortest path from 0 to 5: {' → '.join(map(str, path))}")
print(f"Path length: {len(path) - 1}")

---

## Pattern 3: Cycle Detection

### Detect Cycle in Directed Graph

In [ ]:
def has_cycle_directed(graph):
    """
    Detect cycle in directed graph using DFS.
    Use three states: unvisited, visiting, visited.
    Time: O(V + E), Space: O(V)
    """
    UNVISITED, VISITING, VISITED = 0, 1, 2
    state = {node: UNVISITED for node in graph}
    
    def dfs(node):
        if state[node] == VISITING:
            return True  # Back edge found → cycle!
        
        if state[node] == VISITED:
            return False
        
        state[node] = VISITING
        
        for neighbor in graph[node]:
            if dfs(neighbor):
                return True
        
        state[node] = VISITED
        return False
    
    for node in graph:
        if state[node] == UNVISITED:
            if dfs(node):
                return True
    
    return False

# Example with cycle: 0 → 1 → 2 → 0
graph_cycle = build_graph(3, [(0,1), (1,2), (2,0)], directed=True)
print(f"Graph with cycle: {has_cycle_directed(graph_cycle)}")

# Example without cycle (DAG)
graph_acyclic = build_graph(3, [(0,1), (1,2)], directed=True)
print(f"Graph without cycle: {has_cycle_directed(graph_acyclic)}")

---

## Pattern 4: Topological Sort

**Problem**: Order nodes such that for every edge u → v, u comes before v.

**Requirements**: 
- Only works on Directed Acyclic Graphs (DAG)
- Used for task scheduling, course prerequisites

**Two Methods**:
1. **DFS**: Post-order traversal, reverse result
2. **Kahn's Algorithm**: BFS with in-degree

In [ ]:
def topological_sort_dfs(graph):
    """
    Topological sort using DFS.
    Time: O(V + E), Space: O(V)
    """
    visited = set()
    result = []
    
    def dfs(node):
        visited.add(node)
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor)
        
        result.append(node)  # Add to result after visiting all neighbors
    
    for node in graph:
        if node not in visited:
            dfs(node)
    
    return result[::-1]  # Reverse for correct order

# Example: Course prerequisites
# 0 → 1 means course 0 must be taken before course 1
edges = [(0, 1), (0, 2), (1, 3), (2, 3)]
graph = build_graph(4, edges, directed=True)

order = topological_sort_dfs(graph)
print("Topological order (DFS):", order)
print("Course sequence:", ' → '.join(map(str, order)))

In [ ]:
def topological_sort_kahn(n, edges):
    """
    Kahn's Algorithm: BFS with in-degree.
    Time: O(V + E), Space: O(V + E)
    """
    graph = build_graph(n, edges, directed=True)
    in_degree = {i: 0 for i in range(n)}
    
    # Calculate in-degrees
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1
    
    # Start with nodes having in-degree 0
    queue = deque([node for node in range(n) if in_degree[node] == 0])
    result = []
    
    while queue:
        node = queue.popleft()
        result.append(node)
        
        # Reduce in-degree of neighbors
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    # If result doesn't contain all nodes, there's a cycle
    return result if len(result) == n else None

# Example
edges = [(0, 1), (0, 2), (1, 3), (2, 3)]
order = topological_sort_kahn(4, edges)
print("Topological order (Kahn):", order)
print("Course sequence:", ' → '.join(map(str, order)))

### Visualization: Topological Sort

In [ ]:
def topological_sort_visual(n, edges):
    """Visual walkthrough of Kahn's algorithm."""
    graph = build_graph(n, edges, directed=True)
    in_degree = {i: 0 for i in range(n)}
    
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1
    
    print("Initial state:")
    print(f"  Graph: {dict(graph)}")
    print(f"  In-degrees: {in_degree}\n")
    
    queue = deque([node for node in range(n) if in_degree[node] == 0])
    result = []
    step = 0
    
    print(f"Starting nodes (in-degree 0): {list(queue)}\n")
    
    while queue:
        node = queue.popleft()
        result.append(node)
        step += 1
        
        print(f"Step {step}: Process node {node}")
        print(f"  Current order: {result}")
        
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            print(f"  Reduce in-degree of {neighbor}: {in_degree[neighbor] + 1} → {in_degree[neighbor]}")
            
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
                print(f"    Add {neighbor} to queue (in-degree now 0)")
        
        print(f"  Queue: {list(queue)}\n")
    
    print(f"Final topological order: {result}")
    return result

edges = [(0, 1), (0, 2), (1, 3), (2, 3)]
topological_sort_visual(4, edges)

---

## Example: Course Schedule

**Problem**: Given n courses and prerequisites, can you finish all courses?

In [ ]:
def can_finish(num_courses, prerequisites):
    """
    Check if all courses can be completed (no cycles).
    Use topological sort (Kahn's algorithm).
    Time: O(V + E), Space: O(V + E)
    """
    graph = build_graph(num_courses, prerequisites, directed=True)
    in_degree = {i: 0 for i in range(num_courses)}
    
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1
    
    queue = deque([node for node in range(num_courses) if in_degree[node] == 0])
    count = 0
    
    while queue:
        node = queue.popleft()
        count += 1
        
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    return count == num_courses  # True if no cycle

# Example 1: Can finish
# Course 1 requires course 0
prereqs1 = [(0, 1)]
print(f"Can finish courses {prereqs1}: {can_finish(2, prereqs1)}")

# Example 2: Cannot finish (cycle)
# Course 0 requires 1, and 1 requires 0
prereqs2 = [(0, 1), (1, 0)]
print(f"Can finish courses {prereqs2}: {can_finish(2, prereqs2)}")

# Example 3: Multiple prerequisites
prereqs3 = [(0, 1), (0, 2), (1, 3), (2, 3)]
print(f"Can finish courses {prereqs3}: {can_finish(4, prereqs3)}")

---

## Common Graph Patterns

### 1. DFS Template
```python
def dfs(graph, start):
    visited = set()
    
    def explore(node):
        visited.add(node)
        
        # Process node
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                explore(neighbor)
    
    explore(start)
```

### 2. BFS Template
```python
from collections import deque

def bfs(graph, start):
    visited = {start}
    queue = deque([start])
    
    while queue:
        node = queue.popleft()
        
        # Process node
        
        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
```

### 3. Cycle Detection (Directed)
```python
def has_cycle(graph):
    WHITE, GRAY, BLACK = 0, 1, 2
    color = {node: WHITE for node in graph}
    
    def dfs(node):
        if color[node] == GRAY:
            return True  # Back edge → cycle
        if color[node] == BLACK:
            return False
        
        color[node] = GRAY
        for neighbor in graph[node]:
            if dfs(neighbor):
                return True
        color[node] = BLACK
        return False
    
    return any(dfs(node) for node in graph if color[node] == WHITE)
```

### 4. Topological Sort Template
```python
def topological_sort(n, edges):
    graph = build_graph(n, edges, directed=True)
    in_degree = {i: 0 for i in range(n)}
    
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1
    
    queue = deque([i for i in range(n) if in_degree[i] == 0])
    result = []
    
    while queue:
        node = queue.popleft()
        result.append(node)
        
        for neighbor in graph[node]:
            in_degree[neighbor] -= 1
            if in_degree[neighbor] == 0:
                queue.append(neighbor)
    
    return result if len(result) == n else None
```

---

## Practice Problems

### Easy
1. Number of Connected Components - Count components with DFS/BFS
2. Find if Path Exists - Simple DFS/BFS
3. Clone Graph - Deep copy of graph

### Medium
4. Course Schedule - Detect cycle, topological sort
5. Course Schedule II - Return topological order
6. Number of Islands - DFS on 2D grid
7. Pacific Atlantic Water Flow - Multi-source DFS/BFS
8. Surrounded Regions - Boundary DFS
9. Word Ladder - BFS shortest path
10. Evaluate Division - DFS with weights

### Hard
11. Alien Dictionary - Topological sort from partial order
12. Network Delay Time - Dijkstra's algorithm
13. Critical Connections - Find bridges (Tarjan's)
14. Minimum Height Trees - Find tree centers

## Key Takeaways

- **DFS**: Stack/recursion, depth-first, good for paths and cycles
- **BFS**: Queue, level-by-level, shortest path in unweighted graphs
- **Topological Sort**: Order nodes in DAG (no cycles)
- **Cycle Detection**: Use 3-color DFS for directed graphs
- Most graph algorithms: O(V + E) time
- Adjacency list is most common representation

## DFS vs BFS

| Aspect | DFS | BFS |
|--------|-----|-----|
| **Data Structure** | Stack/Recursion | Queue |
| **Space** | O(h) height | O(w) width |
| **Shortest Path** | No | Yes (unweighted) |
| **Use Case** | Paths, cycles, topological sort | Shortest path, levels |
| **Memory** | Less for wide graphs | Less for deep graphs |

## Graph Algorithm Cheat Sheet

| Problem | Algorithm | Time |
|---------|-----------|------|
| **Traversal** | DFS/BFS | O(V+E) |
| **Shortest Path (unweighted)** | BFS | O(V+E) |
| **Shortest Path (weighted)** | Dijkstra | O(E log V) |
| **Connected Components** | DFS/BFS | O(V+E) |
| **Cycle Detection** | DFS | O(V+E) |
| **Topological Sort** | DFS/Kahn | O(V+E) |
| **Minimum Spanning Tree** | Prim/Kruskal | O(E log V) |

---

**Next**: [Backtracking Pattern](08_backtracking.ipynb)